## 【問題1】コンペティション内容の確認
コンペティションのOverviewページ読み、「Home Credit Default Risk」について以下の観点について確認してください。<br>
　・何を学習し、何を予測するのか<br>
　・どのようなファイルを作りKaggleに提出するか<br>
　・提出されたものはどういった指標値で評価されるのか<br>

・何を学習し、何を予測するのか<br>
　電話会社や取引情報などのさまざまな代替データから学習を行い、クライアントの返済能力を予測する。<br>
・どのようなファイルを作りKaggleに提出するか<br>
　SK_ID_CURRテストセットのそれぞれについて、TARGET変数の確率を予測する必要があります。<br>
　ファイルにはヘッダーが含まれ、次の形式である必要があります。<br>
　　SK_ID_CURR,TARGET<br>
　　100001,0.1<br>
　　100005,0.9<br>
　　100013,0.2<br>
　　etc.<br>
・提出されたものはどういった指標値で評価されるのか<br>
　予測された確率と観察されたターゲットの間のROC曲線の下の領域で評価されます。<br>

## 【問題2】学習と検証
データを簡単に分析、前処理し、学習、検証するまでの一連の流れを作成・実行してください。<br>
検証にはこのコンペティションで使用される評価指標を用いるようにしてください。学習に用いる手法は指定しません。<br>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import auc


# ファイル名（パス）を指定する
csv_path = "E:/DiveIntoCode/source/application_train.csv" # 絶対パス

# 指数表示の禁止を設定する ※numpyのみなので注意
np.set_printoptions(suppress=True)

# csvファイル読み込み ※該当のcsvの１行目が列名となっているため、列名を指定したりせず、そのまま読み込む
df_train = pd.read_csv(csv_path)

# Xに説明変数、yに目的変数となる列をそれぞれ抽出して代入
df_X = df_train[["AMT_INCOME_TOTAL","AMT_CREDIT"]]
df_y = df_train["TARGET"]
# X,yをndarrayへ変換
X = df_X.values
y = df_y.values

# test_splitにX、yを代入（デフォルト(指定なし)で訓練75%、検証25%となる）
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# インスタンス化
scaler = StandardScaler()

# 訓練用のデータのみを.fitで標準化
scaler.fit(X_train)

# 訓練用、検証用双方のデータにtransformで標準化
tr_X_tarin = scaler.transform(X_train)
tr_X_test = scaler.transform(X_test)

# 学習（ランダムフォレスト）
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(tr_X_tarin, y_train)

# 学習モデルによる推定
Y_pred = regr.predict(tr_X_test)

# ROC曲線とAUCの出力
fpr, tpr, thresholds = roc_curve(y_test, Y_pred)
auc_train = auc(fpr, tpr)
# print(roc_curve(y_test, Y_pred))

print("■========== fpr ==========")
print(fpr)
print("■========== tpr ==========")
print(tpr)
print("■========== thresholds ==========")
print(thresholds)
print("■========== auc ==========")
print(auc_train)


■========== fpr ==========
[0.         0.36088673 0.36111308 0.36114137 0.36139601 0.36177798
 0.36191945 0.36271167 0.39168447 0.39171276 0.39316989 0.39418846
 0.44098632 0.44117023 0.44126926 0.44133999 0.44136829 0.44158049
 0.44172196 0.44180684 0.44183513 0.45666106 0.46182466 0.4795224
 0.48416258 0.48417672 0.49426344 0.49625815 0.50139347 0.50171885
 0.50173299 0.50204422 0.50211496 0.50214325 0.50320427 0.503275
 0.50521312 0.50832543 0.50838202 0.51174898 0.51189045 0.51190459
 0.51197533 0.51262608 0.51283829 0.51288073 0.51289487 0.51433786
 0.51481885 0.51486129 0.51562522 0.5157384  0.51593645 0.51641745
 0.51658721 0.51678526 0.51713894 0.51718138 0.51735114 0.51794531
 0.51801604 0.51829898 0.52269866 0.52288257 0.5236182  0.52408505
 0.52415579 0.52424067 0.52429725 0.52640514 0.52643343 0.52660319
 0.52668808 0.52671637 0.52861205 0.52931939 0.52948916 0.5304087
 0.53056432 0.53081896 0.53083311 0.53104531 0.78733006 0.7873725
 0.78738665 0.78875889 0.78880134 0.8376

## 【問題3】テストデータに対する推定
テストデータ（application_test.csv）に対して推定を行い、Kaggleに提出を行ってください。<br>
正しく提出が行えていれば、精度は低くても構いません。<br>

In [2]:
# ファイル名（パス）を指定する
test_csv_path = "E:/DiveIntoCode/source/application_test.csv" # 絶対パス

# 指数表示の禁止を設定する ※numpyのみなので注意
np.set_printoptions(suppress=True)

# csvファイル読み込み ※該当のcsvの１行目が列名となっているため、列名を指定したりせず、そのまま読み込む
test_df_train = pd.read_csv(test_csv_path)

# Xに説明変数、yに目的変数となる列をそれぞれ抽出して代入
test_df_X = test_df_train[["AMT_INCOME_TOTAL","AMT_CREDIT"]]
# X,yをndarrayへ変換
test_X = test_df_X.values

# インスタンス化
test_scaler = StandardScaler()

# 訓練用のデータのみを.fitで標準化
test_scaler.fit(test_X)

# 訓練用、検証用双方のデータにtransformで標準化
test_tr_X = scaler.transform(test_X)

# 学習モデルによる推定
test_Y_pred = regr.predict(test_tr_X)

### 一度出力しているためコメントアウト
# 提出用に「TARGET」としてデータセットへ組み込む
# test_df_train["TARGET"] = np.around(test_Y_pred, 1)
# 提出に必要な列「SK_ID_CURR」と「TARGET」を抽出
# df_for_submission = test_df_train[["SK_ID_CURR","TARGET"]]
# csvへ出力
# df_for_submission.to_csv("submission.csv", index=False)


## 【問題4】特徴量エンジニアリング
精度を上げるために以下のような観点で 特徴量エンジニアリング（Feature Engineering） を行ってください。<br>
　・どの特徴量を使うか<br>
　・どう前処理をするか<br>
何をした時に検証データに対する評価指標がどのようになったかをまとめてください。最低5パターンの学習・検証を行ってください。<br>

In [3]:
# ファイル名（パス）を指定する
csv_path = "E:/DiveIntoCode/source/application_train.csv" # 絶対パス

# 指数表示の禁止を設定する ※numpyのみなので注意
np.set_printoptions(suppress=True)

# csvファイル読み込み ※該当のcsvの１行目が列名となっているため、列名を指定したりせず、そのまま読み込む
df_train = pd.read_csv(csv_path)

# Xに説明変数、yに目的変数となる列をそれぞれ抽出し、ndarrayへ変換して、変数へ代入
# 使用した指標は次の3つ（3つの指標の組み合わせに対してその後の前処理、学習を行うため複数用意）
# AMT_INCOME_TOTAL：クライアントの収入
# AMT_CREDIT：クライアントの借金の総額
# DAYS_EMPLOYED：申請の何日前にその人が現在の雇用を開始したか、申請に関連する時間のみ
df_X_1 = df_train[["AMT_INCOME_TOTAL","AMT_CREDIT"]].values
df_X_2 = df_train[["AMT_INCOME_TOTAL","DAYS_EMPLOYED"]].values
df_X_3 = df_train[["AMT_CREDIT","DAYS_EMPLOYED"]].values
df_y = df_train["TARGET"].values

# (デフォルト(指定なし)で訓練75%、検証25%となる）
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(df_X_1, y, random_state=42)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(df_X_2, y, random_state=42)
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(df_X_3, y, random_state=42)

# インスタンス化　※標準化
stdsc = StandardScaler()
# 訓練用のデータはfit→transform、検証用のデータはtransformのみ
X_train_1_stdsc = stdsc.fit_transform(X_train_1)
X_train_2_stdsc = stdsc.fit_transform(X_train_2)
X_train_3_stdsc = stdsc.fit_transform(X_train_3)
X_test_1_stdsc = stdsc.transform(X_test_1)
X_test_2_stdsc = stdsc.transform(X_test_2)
X_test_3_stdsc = stdsc.transform(X_test_3)

# インスタンス化　※正規化
mmsc = MinMaxScaler()
# 訓練用のデータはfit→transform、検証用のデータはtransformのみ
X_train_1_mmsc = mmsc.fit_transform(X_train_1)
X_train_2_mmsc = mmsc.fit_transform(X_train_2)
X_train_3_mmsc = mmsc.fit_transform(X_train_3)
X_test_1_mmsc = mmsc.transform(X_test_1)
X_test_2_mmsc = mmsc.transform(X_test_2)
X_test_3_mmsc = mmsc.transform(X_test_3)


In [4]:
### 学習で時間がかかるので分割

# ランダムフォレスト モデルの作成分だけインスタンス化
regr_stdsc_1 = RandomForestRegressor(max_depth=2, random_state=42)
regr_stdsc_2 = RandomForestRegressor(max_depth=2, random_state=42)
regr_stdsc_3 = RandomForestRegressor(max_depth=2, random_state=42)
regr_mmsc_1 = RandomForestRegressor(max_depth=2, random_state=42)
regr_mmsc_2 = RandomForestRegressor(max_depth=2, random_state=42)
regr_mmsc_3 = RandomForestRegressor(max_depth=2, random_state=42)

# 学習
regr_stdsc_1.fit(X_train_1_stdsc, y_train)
regr_stdsc_2.fit(X_train_2_stdsc, y_train)
regr_stdsc_3.fit(X_train_3_stdsc, y_train)
regr_mmsc_1.fit(X_train_1_mmsc, y_train)
regr_mmsc_2.fit(X_train_2_mmsc, y_train)
regr_mmsc_3.fit(X_train_3_mmsc, y_train)


RandomForestRegressor(max_depth=2, random_state=42)

In [5]:
### 学習で時間がかかるので分割

# 学習モデルによる推定
Y_pred_stdsc_1 = regr_stdsc_1.predict(X_test_1_stdsc)
Y_pred_stdsc_2 = regr_stdsc_2.predict(X_test_2_stdsc)
Y_pred_stdsc_3 = regr_stdsc_3.predict(X_test_3_stdsc)
Y_pred_mmsc_1 = regr_mmsc_1.predict(X_test_1_mmsc)
Y_pred_mmsc_2 = regr_mmsc_2.predict(X_test_2_mmsc)
Y_pred_mmsc_3 = regr_mmsc_3.predict(X_test_3_mmsc)

# ROC曲線とAUCの出力
fpr_stdsc_1, tpr_stdsc_1, thresholds_stdsc_1 = roc_curve(y_test, Y_pred_stdsc_1)
fpr_stdsc_2, tpr_stdsc_2, thresholds_stdsc_2 = roc_curve(y_test, Y_pred_stdsc_2)
fpr_stdsc_3, tpr_stdsc_3, thresholds_stdsc_3 = roc_curve(y_test, Y_pred_stdsc_3)
fpr_mmsc_1, tpr_mmsc_1, thresholds_mmsc_1 = roc_curve(y_test, Y_pred_mmsc_1)
fpr_mmsc_2, tpr_mmsc_2, thresholds_mmsc_2 = roc_curve(y_test, Y_pred_mmsc_2)
fpr_mmsc_3, tpr_mmsc_3, thresholds_mmsc_3 = roc_curve(y_test, Y_pred_mmsc_3)

# print(fpr_stdsc_1)
# print(type(fpr_stdsc_1))
# print(fpr_mmsc_1)
# print(type(fpr_mmsc_1))

auc_stdsc_1 = auc(fpr_stdsc_1, tpr_stdsc_1)
auc_stdsc_2 = auc(fpr_stdsc_2, tpr_stdsc_2)
auc_stdsc_3 = auc(fpr_stdsc_3, tpr_stdsc_3)
auc_mmsc_1 = auc(fpr_mmsc_1, tpr_mmsc_1)
auc_mmsc_2 = auc(fpr_mmsc_2, tpr_mmsc_2)
auc_mmsc_3 = auc(fpr_mmsc_3, tpr_mmsc_3)

# AMT_INCOME_TOTAL：クライアントの収入
# AMT_CREDIT：クライアントの借金の総額
# DAYS_EMPLOYED：申請の何日前にその人が現在の雇用を開始したか、申請に関連する時間のみ

print("{}　：【特徴量】「AMT_INCOME_TOTAL」「AMT_CREDIT」、【前処理】標準化".format(auc_stdsc_1))
print("{}　：【特徴量】「AMT_INCOME_TOTAL」「DAYS_EMPLOYED」、【前処理】標準化".format(auc_stdsc_2))
print("{}　：【特徴量】「AMT_CREDIT」「DAYS_EMPLOYED」、【前処理】標準化".format(auc_stdsc_3))
print("{}　：【特徴量】「AMT_INCOME_TOTAL」「AMT_CREDIT」、【前処理】正規化".format(auc_mmsc_1))
print("{}　：【特徴量】「AMT_INCOME_TOTAL」「DAYS_EMPLOYED」、【前処理】正規化".format(auc_mmsc_2))
print("{}　：【特徴量】「AMT_CREDIT」「DAYS_EMPLOYED」、【前処理】正規化".format(auc_mmsc_3))


0.5019517942988844　：【特徴量】「AMT_INCOME_TOTAL」「AMT_CREDIT」、【前処理】標準化
0.5885483539142302　：【特徴量】「AMT_INCOME_TOTAL」「DAYS_EMPLOYED」、【前処理】標準化
0.5897711478136682　：【特徴量】「AMT_CREDIT」「DAYS_EMPLOYED」、【前処理】標準化
0.5005492258880772　：【特徴量】「AMT_INCOME_TOTAL」「AMT_CREDIT」、【前処理】正規化
0.588809484941015　：【特徴量】「AMT_INCOME_TOTAL」「DAYS_EMPLOYED」、【前処理】正規化
0.5897706850868472　：【特徴量】「AMT_CREDIT」「DAYS_EMPLOYED」、【前処理】正規化


| 特徴量の組合せ | 前処理 | 結果 |
|:-|:-:|-:|
| AMT_INCOME_TOTAL、AMT_CREDIT | 標準化 | 0.5019517942988844 |
| AMT_INCOME_TOTAL、DAYS_EMPLOYED | 標準化 | 0.5885483539142302 |
| AMT_CREDIT、DAYS_EMPLOYED | 標準化 | 0.5897711478136682 |
| AMT_INCOME_TOTAL、AMT_CREDIT | 正規化 | 0.5005492258880772 |
| AMT_INCOME_TOTAL、DAYS_EMPLOYED | 正規化 | 0.5888094849410150 |
| AMT_CREDIT、DAYS_EMPLOYED | 正規化 | 0.5897706850868472 |

<< 考察 >><br>
特徴量(3種類)と前処理(2種類)の組合せを行い、ランダムフォレスト(パラメータはデフォルト)による学習を行った。<br>
結果、特徴量の組合せに注目すると、精度が上がっており、前処理に注目すると大きな違いは見られなかった。<br>
今回は前処理で大きな変化は見られなかったが、特徴量や前処理の仕方によっては結果に大きな違いが生まれてくることが分かった。<br>
学習方法やそのパラメータとの組合せを考えると膨大な量で検証を行う必要がある。<br>
それを全部行うことはできない(と思われる)ため、特徴量の選出、前処理、学習の各工程で適切な判断が必要となる。<br>
そのためにも、各工程で何を行っているのかを理解し、臨機応変な対応が出来るようになっておきたい。<br>

In [6]:
"""

以下、ランダムフォレストのパラメータを弄る場合

組み合わせ
前処理｜ランダムフォレストのパラメータ n_estimators = 100(デフォルト)、min_samples_split = 2(デフォルト)、random_state=42（固定）
---------------------------------------
標準化｜ランダムフォレスト n_estimators = 100、max_depth = 2、min_samples_split = 2、random_state=42
標準化｜ランダムフォレスト n_estimators = 1000、max_depth = 2、min_samples_split = 2、random_state=42
標準化｜ランダムフォレスト n_estimators = 100、max_depth = 10、min_samples_split = 2、random_state=42
標準化｜ランダムフォレスト n_estimators = 100、max_depth = 2、min_samples_split = 10、random_state=42

正規化｜ランダムフォレスト n_estimators = 100、max_depth = 2、min_samples_split = 2、random_state=42
正規化｜ランダムフォレスト n_estimators = 1000、max_depth = 2、min_samples_split = 2、random_state=42
正規化｜ランダムフォレスト n_estimators = 100、max_depth = 10、min_samples_split = 2、random_state=42
正規化｜ランダムフォレスト n_estimators = 100、max_depth = 2、min_samples_split = 10、random_state=42

"""


'\n\n以下、ランダムフォレストのパラメータを弄る場合\n\n組み合わせ\n前処理｜ランダムフォレストのパラメータ n_estimators = 100(デフォルト)、min_samples_split = 2(デフォルト)、random_state=42（固定）\n---------------------------------------\n標準化｜ランダムフォレスト n_estimators = 100、max_depth = 2、min_samples_split = 2、random_state=42\n標準化｜ランダムフォレスト n_estimators = 1000、max_depth = 2、min_samples_split = 2、random_state=42\n標準化｜ランダムフォレスト n_estimators = 100、max_depth = 10、min_samples_split = 2、random_state=42\n標準化｜ランダムフォレスト n_estimators = 100、max_depth = 2、min_samples_split = 10、random_state=42\n\n正規化｜ランダムフォレスト n_estimators = 100、max_depth = 2、min_samples_split = 2、random_state=42\n正規化｜ランダムフォレスト n_estimators = 1000、max_depth = 2、min_samples_split = 2、random_state=42\n正規化｜ランダムフォレスト n_estimators = 100、max_depth = 10、min_samples_split = 2、random_state=42\n正規化｜ランダムフォレスト n_estimators = 100、max_depth = 2、min_samples_split = 10、random_state=42\n\n'